In [2]:
from mongoengine import *
import os
import pandas as pd
from pandas import Series,DataFrame
import datetime

In [3]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [4]:

class Extremum(EmbeddedDocument):
    '''
        极值
    '''
    occurredTime = DateTimeField(required=False, default=None)
    val = IntField(required=False, default=None)

    # def __init__(self,date:datetime,val:str):
    #     # self.occurredTime=date
    #     # self.val=val
    #     self._toModel(date,val)
    #
    # def _toModel(self,date:datetime,val:str):
    #     '''
    #         加入一个数据验证
    #     :param date:
    #     :param val:
    #     :return:
    #     '''
    #     occurredTime= None if (date is '--' or date is None) else date
    #     val=None if (val is '--' or val is None) else val


class ForecastData(EmbeddedDocument):
    # 极大值的24小时观测数组
    forecast_arr = ListField(IntField(default=None))
    heigh_heigh_tide = EmbeddedDocumentField(Extremum)
    heigh_low_tide = EmbeddedDocumentField(Extremum)
    low_heigh_tide = EmbeddedDocumentField(Extremum)
    low_low_tide = EmbeddedDocumentField(Extremum)


class RealData(EmbeddedDocument):
    # 极大值的24小时观测数组
    realdata_arr = ListField(IntField(default=None), default=None)
    heigh_heigh_tide = EmbeddedDocumentField(Extremum)
    heigh_low_tide = EmbeddedDocumentField(Extremum)
    low_heigh_tide = EmbeddedDocumentField(Extremum)
    low_low_tide = EmbeddedDocumentField(Extremum)


class TideData(EmbeddedDocument):
    '''
        测站数据
    '''
    # 极大值的24小时观测数组
    # forecast_arr = ListField(IntField())
    # # 极小值得24小时观测数组
    # realdata_arr = ListField(IntField())
    # 目标日期（年-月-日）
    #     targetdate=DateTimeField()
    targetdate = DateField(default=None)
    forecastdata = EmbeddedDocumentField(ForecastData)
    realdata = EmbeddedDocumentField(RealData)
    # heigh_heigh_tide = EmbeddedDocumentField(Extremum)
    # heigh_low_tide = EmbeddedDocumentField(Extremum)
    # low_heigh_tide = EmbeddedDocumentField(Extremum)
    # low_low_tide = EmbeddedDocumentField(Extremum)


class StationTideData(Document):
    '''
        测站潮位数据
    '''
    # 测站代码
    code = StringField(max_length=10)
    #     起始时间
    startdate = DateTimeField()
    #     测站名称
    stationname = StringField(max_length=10)
    #     gesjon数据
    point = PointField()
    #     平均海平面
    lev = IntField()
    #     警戒潮位
    jw = IntField()
    #     潮汐调和常数
    harmonicconstant = StringField()
    #     潮位数据
    realtidedata = ListField(EmbeddedDocumentField(TideData))
    # tideDataMin = EmbeddedDocumentField(TideData)
    meta = {'collection': 'geostationtidedata'}

## 1 测试查询使用

In [5]:
filter_list=StationTideData.objects(code='5622')

In [6]:
filter_list

[<StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>, <StationTideData: StationTideData object>]

In [21]:
t1='1956-09-04 00:00'

In [22]:
targetdate=datetime.datetime.strptime(t1,'%Y-%m-%d %H:%M')
filter_list=StationTideData.objects(code='5622',startdate=targetdate)

In [23]:
len(filter_list)

4

In [24]:
for temp in filter_list:
    print(temp.startdate)

1956-09-04 00:00:00
1956-09-04 00:00:00
1956-09-04 00:00:00
1956-09-04 00:00:00


In [17]:
filter_list[:1][0].startdate

datetime.datetime(1956, 9, 2, 0, 0)

In [25]:
filter_list[:1][0].realtidedata

[<TideData: TideData object>,
 <TideData: TideData object>,
 <TideData: TideData object>,
 <TideData: TideData object>]

## 2

In [22]:
import dateutil
from datetime import datetime,timedelta,date

In [17]:
date_str='1956-09-03 00:00:00+00:00'
code='5622'

In [18]:
targetdate = dateutil.parser.parse(date_str)
targetdate

datetime.datetime(1956, 9, 3, 0, 0, tzinfo=tzutc())

In [24]:
date(1956,9,3)

datetime.date(1956, 9, 3)

In [28]:
list=StationTideData.objects(code=code,startdate=date(1956,9,3))

In [29]:
for temp in list:
    print(temp.startdate)

1956-09-03 00:00:00
1956-09-03 00:00:00
1956-09-03 00:00:00
1956-09-03 00:00:00


In [31]:
# list(list)